In [8]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "WH"
BARK_VAR = "WC"
FIA_SPCD = 263

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(0.0, 0.93371, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,11023.000000,1.102300e+04,11023.0,11023.000000,11023.000000,11023.000000,11023.000000,11023.000000,11023.000000,11023.000000,...,450.000000,450.000000,11023.000000,11023.000000,11023.000000,11023.000000,11023.0,11023.000000,11023.000000,11023.000000
mean,617.956545,4.837046e+11,263.0,139.067991,0.385094,3.096250,24.820376,14.345686,0.530373,0.487965,...,0.268667,0.586444,2005.347183,7.941032,2015.347183,7.397532,10.0,2.536152,9.183707,859.573347
std,59.904642,6.258495e+10,0.0,58.574204,0.257229,1.947415,12.218109,10.382984,0.200821,0.192369,...,0.215972,0.457059,2.430868,1.738212,2.430868,1.734145,0.0,0.799618,5.844357,497.952252
min,113.000000,6.010155e+10,263.0,4.400000,0.000000,0.000000,1.000000,5.000000,0.030000,0.010000,...,0.050000,0.050000,2001.000000,1.000000,2011.000000,3.000000,10.0,0.000000,0.000000,0.000000
25%,605.000000,4.101042e+11,263.0,100.414000,0.170000,1.326450,15.000000,7.100000,0.400000,0.350000,...,0.100000,0.250000,2003.000000,7.000000,2013.000000,6.000000,10.0,2.000000,6.000000,472.000000
50%,609.000000,5.305058e+11,263.0,142.500000,0.350000,3.298672,27.000000,10.300000,0.500000,0.450000,...,0.200000,0.500000,2005.000000,8.000000,2015.000000,7.000000,10.0,3.000000,9.000000,790.000000
75%,617.000000,5.306032e+11,263.0,176.208000,0.600000,4.764749,35.000000,17.000000,0.680000,0.650000,...,0.350000,0.800000,2007.000000,9.000000,2017.000000,9.000000,10.0,3.000000,14.000000,1277.000000
max,800.000000,5.309066e+11,263.0,336.516000,1.500000,6.283185,55.000000,91.000000,0.990000,0.980000,...,2.050000,4.150000,2009.000000,12.000000,2019.000000,11.000000,10.0,6.000000,23.000000,1776.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [19:53<00:00,  1.68it/s, 255 steps of size 2.12e-02. a


Done.
Samples saved at ../../models/maicf/WH//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/WH//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [31:57<00:00,  1.04it/s, 255 steps of size 1.74e-02. a


Done.
Samples saved at ../../models/maicf/WH//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/WH//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/WH//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/WH//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [19:52<00:00,  1.68it/s, 255 steps of size 2.09e-02. a


Done.
Samples saved at ../../models/maicf/WH//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/WH//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [32:03<00:00,  1.04it/s, 255 steps of size 1.69e-02. a


Done.
Samples saved at ../../models/maicf/WH//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/WH//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [9]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [15:14<00:00,  2.19it/s, 255 steps of size 1.92e-02. a


Done.
Samples saved at ../../models/maicf/WH//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/WH//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [15:23<00:00,  2.17it/s, 255 steps of size 1.89e-02. a


Done.
Samples saved at ../../models/maicf/WH//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/WH//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/WH//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/WH//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [22:45<00:00,  1.46it/s, 255 steps of size 1.85e-02. a


Done.
Samples saved at ../../models/maicf/WH//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/WH//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [23:09<00:00,  1.44it/s, 255 steps of size 1.81e-02. a


Done.
Samples saved at ../../models/maicf/WH//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/WH//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
